In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import (
    f1_score, 
    precision_score, 
    recall_score, 
    accuracy_score, 
    classification_report
)
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle

In [2]:
data = pd.read_pickle("/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/plbart_base_embeddings/processed_embeddings/plbart_embed_line_by_line_sum.pkl")
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,45.869217,-159.518372,23.207367,-137.124603,128.811096,-82.769165,-81.768135,-0.640440,-128.941055,-85.681137,...,81.719536,111.293434,-95.435867,-221.365875,-222.611801,117.390900,-35.564804,1,12150,critical
1,3.429673,-8.777965,4.491210,-10.657660,9.625399,-4.418340,3.721420,-7.408656,-4.963110,-15.298643,...,7.573229,5.523479,-7.450909,-16.093414,-12.905525,13.912349,8.611141,1,10168,critical
2,247.751678,-129.334778,-16.591146,-173.292419,301.242645,-194.701111,-162.284851,-9.360210,-156.371552,-213.367981,...,124.247910,92.825050,-123.665474,-353.930145,-335.222412,256.613007,18.244608,1,12783,critical
3,65.134140,-127.012352,-15.722034,-256.294617,304.530762,-158.203156,-216.614334,-7.443196,-264.936035,-183.173889,...,113.942245,72.379677,-109.332008,-295.930847,-333.070648,286.990631,-100.377357,1,12754,critical
4,125.089897,-107.790710,18.801786,-104.372971,193.609970,-143.885544,-98.533447,15.337610,-114.026794,-96.366798,...,83.062164,54.470287,-99.624298,-274.242889,-184.718567,147.438492,6.238143,1,4277,critical


In [3]:
# Separate features and labels
X = data[[col for col in data.columns if col.startswith("emb_")]].values  # Select embedding columns
y = data['label'].values 

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define classifiers
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
bagging = BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=100, random_state=42)  
xgboost = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Add the classifiers to a dictionary
classifiers = {
    "Random Forest": random_forest,
    "Bagging": bagging,
    "XGBoost": xgboost
}

In [4]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store cross-validation results
results = {}

# Perform cross-validation for each classifier
for clf_name, clf in classifiers.items():
    metrics_summary = {
        "f1_micro": [],
        "f1_macro": [],
        "accuracy": [],
        "precision_macro": [],
        "recall_macro": []
    }
    
    for train_idx, val_idx in skf.split(X_train, y_train):
        # Split the training data into training and validation folds
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        # Train the classifier
        clf.fit(X_train_fold, y_train_fold)
        
        # Make predictions on the validation set
        y_pred = clf.predict(X_val_fold)
        
        # Calculate metrics
        metrics_summary["f1_micro"].append(f1_score(y_val_fold, y_pred, average='micro'))
        metrics_summary["f1_macro"].append(f1_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["accuracy"].append(accuracy_score(y_val_fold, y_pred))
        metrics_summary["precision_macro"].append(precision_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["recall_macro"].append(recall_score(y_val_fold, y_pred, average='macro'))
    
    # Store mean metrics for the classifier
    results[clf_name] = {metric: np.mean(scores) for metric, scores in metrics_summary.items()}

# Print cross-validation results
print("\nCross-Validation Results:")
for clf_name, metrics in results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")


Cross-Validation Results:

Random Forest Metrics:
F1_micro: 0.8310
F1_macro: 0.5818
Accuracy: 0.8310
Precision_macro: 0.6022
Recall_macro: 0.5731

Bagging Metrics:
F1_micro: 0.8451
F1_macro: 0.5625
Accuracy: 0.8451
Precision_macro: 0.6118
Recall_macro: 0.5535

XGBoost Metrics:
F1_micro: 0.8300
F1_macro: 0.5696
Accuracy: 0.8300
Precision_macro: 0.5901
Recall_macro: 0.5624


In [5]:
# Final evaluation on the test set
final_results = {}

for clf_name, clf in classifiers.items():
    # Train the classifier on the entire training set
    clf.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_test = clf.predict(X_test)
    
    # Calculate test metrics
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    precision_test = precision_score(y_test, y_pred_test, average='macro')
    recall_test = recall_score(y_test, y_pred_test, average='macro')
    accuracy_test = accuracy_score(y_test, y_pred_test)
    report_test = classification_report(y_test, y_pred_test)
    
    # Save final metrics for the classifier
    final_results[clf_name] = {
        "f1_micro": f1_micro_test,
        "f1_macro": f1_macro_test,
        "precision_macro": precision_test,
        "recall_macro": recall_test,
        "accuracy": accuracy_test,
        "classification_report": report_test
    }

# Print final test results
print("\nFinal Test Results:")
for clf_name, metrics in final_results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        if metric != "classification_report":
            print(f"{metric.capitalize()}: {value:.4f}")
    print(f"Classification Report:\n{metrics['classification_report']}")


Final Test Results:

Random Forest Metrics:
F1_micro: 0.8196
F1_macro: 0.5760
Precision_macro: 0.5858
Recall_macro: 0.5699
Accuracy: 0.8196
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.91      0.90       786
           1       0.29      0.23      0.25       123

    accuracy                           0.82       909
   macro avg       0.59      0.57      0.58       909
weighted avg       0.80      0.82      0.81       909


Bagging Metrics:
F1_micro: 0.8449
F1_macro: 0.5674
Precision_macro: 0.6119
Recall_macro: 0.5571
Accuracy: 0.8449
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.95      0.91       786
           1       0.34      0.16      0.22       123

    accuracy                           0.84       909
   macro avg       0.61      0.56      0.57       909
weighted avg       0.81      0.84      0.82       909


XGBoost Metrics:
F1_micro: 0.8251
F1_macro: 

In [9]:
output_path = "/Users/mac/Desktop/Code_Smell_Detection/feature_envy/results/plbart/result_plbart_sum.pkl"
with open(output_path, "wb") as f:
    pickle.dump({"cross_val_results": results, "final_results": final_results}, f)
print(f"\nResults saved to {output_path}")


Results saved to /Users/mac/Desktop/Code_Smell_Detection/feature_envy/results/plbart/result_plbart_sum.pkl
